In [94]:
import polars as pl
import mlxtend.frequent_patterns as fp
import mlxtend.preprocessing as pp
import pickle
import pandas as pd
from efficient_apriori import apriori as apriori_efficient
import numpy as np
import pyarrow as pa

In [95]:
data = pl.read_csv('data/bgg-26m-reviews.csv')
print(data.head())

shape: (5, 6)
┌─────┬─────────────┬────────┬─────────────────────────────────┬─────┬───────┐
│     ┆ user        ┆ rating ┆ comment                         ┆ ID  ┆ name  │
│ --- ┆ ---         ┆ ---    ┆ ---                             ┆ --- ┆ ---   │
│ i64 ┆ str         ┆ f64    ┆ str                             ┆ i64 ┆ str   │
╞═════╪═════════════╪════════╪═════════════════════════════════╪═════╪═══════╡
│ 0   ┆ sidehacker  ┆ 10.0   ┆ null                            ┆ 13  ┆ CATAN │
│ 1   ┆ Varthlokkur ┆ 10.0   ┆ null                            ┆ 13  ┆ CATAN │
│ 2   ┆ dougthonus  ┆ 10.0   ┆ Currently, this sits on my lis… ┆ 13  ┆ CATAN │
│ 3   ┆ cypar7      ┆ 10.0   ┆ I know it says how many plays,… ┆ 13  ┆ CATAN │
│ 4   ┆ ssmooth     ┆ 10.0   ┆ null                            ┆ 13  ┆ CATAN │
└─────┴─────────────┴────────┴─────────────────────────────────┴─────┴───────┘


## Filtering

In [96]:
data = data.filter(pl.col('rating') >= 8)
print(data.head())

shape: (5, 6)
┌─────┬─────────────┬────────┬─────────────────────────────────┬─────┬───────┐
│     ┆ user        ┆ rating ┆ comment                         ┆ ID  ┆ name  │
│ --- ┆ ---         ┆ ---    ┆ ---                             ┆ --- ┆ ---   │
│ i64 ┆ str         ┆ f64    ┆ str                             ┆ i64 ┆ str   │
╞═════╪═════════════╪════════╪═════════════════════════════════╪═════╪═══════╡
│ 0   ┆ sidehacker  ┆ 10.0   ┆ null                            ┆ 13  ┆ CATAN │
│ 1   ┆ Varthlokkur ┆ 10.0   ┆ null                            ┆ 13  ┆ CATAN │
│ 2   ┆ dougthonus  ┆ 10.0   ┆ Currently, this sits on my lis… ┆ 13  ┆ CATAN │
│ 3   ┆ cypar7      ┆ 10.0   ┆ I know it says how many plays,… ┆ 13  ┆ CATAN │
│ 4   ┆ ssmooth     ┆ 10.0   ┆ null                            ┆ 13  ┆ CATAN │
└─────┴─────────────┴────────┴─────────────────────────────────┴─────┴───────┘


## Grouping into baskets

In [137]:
baskets = (
    data
    .group_by(['user'])
    .agg(
        pl.col('name').alias('games')
    )
)

### Training and test split

In [143]:
baskets_train = baskets.head(int(0.9 * len(baskets)))
baskets_test = baskets.tail(len(baskets) - int(0.9 * len(baskets)))
baskets_test

user,games
str,list[str]
"""xyplex64""","[""Prophecy""]"
"""oliverklee""","[""Codenames"", ""Splendor"", … ""Privacy Quickie""]"
"""strayllama""","[""HeroQuest Advanced Quest""]"
"""tpir72""","[""7 Wonders"", ""Twilight Struggle"", … ""Axis Empires: Dai Senso! – World War II in Asia & the Pacific, 1937-45""]"
"""BoardsNMeeples""","[""Satori""]"
…,…
"""jso69""","[""Vikingjarl""]"
"""Conetund70""","[""Ticket to Ride: Europe"", ""Dominion"", … ""Cornish Smuggler""]"
"""Lizzie1012""","[""Love Letter"", ""CATAN"", … ""Fish Frenzy""]"


In [144]:
baskets_test = baskets_test.filter(pl.col('games').list.len() >= 3)
baskets_test = baskets_test.to_pandas()

baskets_test['test'] = baskets_test['games'].apply(lambda x: x[0:int(np.ceil(0.3*len(x)))])
baskets_test['games'] = baskets_test['games'].apply(lambda x: x[int(np.ceil(0.3*len(x))):])

In [145]:
baskets_test_training = baskets_test[['user', 'games']]

In [146]:
baskets_train = baskets_train.to_pandas()
baskets_train = pd.concat([baskets_train, baskets_test_training], ignore_index=True)

In [147]:
baskets_train.shape

(520165, 2)

## Getting frequent itemsets

In [106]:
baskets_train_as_tuples = [tuple(row) for row in baskets_train['games']]
itemsets, rules = apriori_efficient(baskets_train_as_tuples, min_support=0.005, min_confidence=0.7)

In [107]:
pickle.dump((itemsets, rules), open('data/freq_itemsets_approach2_0.005_0.8.pkl', 'wb'))

In [108]:
itemsets, rules = pickle.load(open('data/freq_itemsets_approach2_0.005_0.8.pkl', 'rb'))

In [109]:
len_transactions = len(baskets_train_as_tuples)
itemsets_flattened = []
supports = []
for i in itemsets.keys():
    itemsets_flattened.extend([item[0] for item in itemsets[i].items()])
    supports.extend([item[1]/len_transactions for item in itemsets[i].items()])

In [110]:
itemsets_dict = {
    'support': supports,
    'itemsets': itemsets_flattened
}
itemsets_df = pd.DataFrame(itemsets_dict)
itemsets_df.head()

,support,itemsets
0,0.086005,"(Agricola,)"
1,0.119107,"(Pandemic,)"
2,0.085069,"(Puerto Rico,)"
3,0.070896,"(Ticket to Ride: Europe,)"
4,0.075152,"(Power Grid,)"


## Association rules

In [111]:
rules = fp.association_rules(itemsets_df, metric="confidence", min_threshold=0.6)
rules.tail()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
15705,"(Ark Nova, Scythe, Wingspan, Lost Ruins of Arn...",(Terraforming Mars),0.006422,0.147759,0.005288,0.823423,5.572744,1.0,0.004339,4.826449,0.825859,0.035515,0.792808,0.429605
15706,"(Ark Nova, Scythe, Wingspan, Terraforming Mars...",(Lost Ruins of Arnak),0.007842,0.062629,0.005288,0.674287,10.766312,1.0,0.004797,2.877907,0.914288,0.081125,0.652525,0.379360
15707,"(Ark Nova, Scythe, Lost Ruins of Arnak, Terraf...",(Wingspan),0.007433,0.125615,0.005288,0.711466,5.663863,1.0,0.004354,3.030442,0.829608,0.041390,0.670015,0.376782
15708,"(Ark Nova, Wingspan, Lost Ruins of Arnak, Terr...",(Scythe),0.007835,0.113744,0.005288,0.674889,5.933380,1.0,0.004397,2.726006,0.838028,0.045472,0.633163,0.360689
15709,"(Scythe, Wingspan, Lost Ruins of Arnak, Terraf...",(Ark Nova),0.006955,0.074388,0.005288,0.760295,10.220628,1.0,0.004771,3.861456,0.908477,0.069528,0.741030,0.415691


## Recommendation function

In [112]:
def recommender_association(rules_df, product_list, N=1):
    candidate_rules = rules_df[rules_df['antecedents'].apply(lambda x: x.issubset(product_list))]
    candidate_rules = candidate_rules.sort_values("confidence")
    recommendation_list = []
    for i in range(len(candidate_rules)):
        for item in candidate_rules.iloc[i]['consequents']:
            if item not in product_list and item not in recommendation_list:
                recommendation_list.append(item)
            if len(recommendation_list) >= N:
                break
        if len(recommendation_list) >= N:
            break

    return recommendation_list

## Evaluation

In [118]:
predictions = {
    'user': [],
    'recommended_items': [],
    'true_items': []
}

for index, row in baskets_test.iterrows():
    user = row['user']
    train_items = row['games']
    test_items = row['test']
    recommended_items = recommender_association(rules, train_items, N=5)

    predictions['user'].append(user)
    predictions['recommended_items'].append(recommended_items)
    predictions['true_items'].append(test_items)
    
predictions_df = pd.DataFrame(predictions)
predictions_df.head()

,user,recommended_items,true_items
0,Kiperius,"[Agricola, Puerto Rico]","[Splendor, King of Tokyo, Small World, Ticket ..."
1,aurora1986,[Small World],"[Small World, Ticket to Ride, Pandemic, Domini..."
2,bigcpwnzj00,[],"[Agricola, Splendor, Terraforming Mars, Root, ..."
3,RaynorGX,[],[Pandemic]
4,Fatmansky,[],"[CATAN, Terraforming Mars]"


In [119]:
pickle.dump(predictions_df, open('data/freq_itemsets_pred_df.pkl', 'wb'))

In [120]:
predictions_df = pickle.load(open('data/freq_itemsets_pred_df.pkl', 'rb'))

In [125]:
predictions_df.sample(20)

,user,recommended_items,true_items
52675,Mozbie,[],"[Splendor, Hanabi]"
70720,Lillybet118,"[Wingspan, Ark Nova, Gloomhaven, Terraforming ...","[Love Letter, Azul, Wingspan, 7 Wonders Duel, ..."
41570,ProfesssorPeverell,"[7 Wonders Duel, Agricola, Azul, Terraforming ...","[Small World, Ticket to Ride, Azul, Agricola, ..."
67350,tennisrie,[],"[Dominion, Pandemic, Forbidden Island, Citadel..."
64151,Manini,[],[Star Wars: Destiny]
66934,azulazim,[],"[Mage Knight Board Game, Sword & Sorcery, Batt..."
31810,Hazebot,[],"[Forbidden Island, Pandemic Legacy: Season 1]"
41813,horbinom,"[The Castles of Burgundy, 7 Wonders Duel]","[Scythe, 7 Wonders Duel]"
19389,cByte,[],"[Splendor, Azul, Wingspan, Ticket to Ride: Eur..."
57490,sambfwin,[Puerto Rico],"[Agricola, Pandemic, Puerto Rico, Small World,..."


In [132]:
## Precision
def precision_at_k(recommended_items, true_items, k):
    recommended_at_k = recommended_items if len(recommended_items) < k else recommended_items[:k]
    true_positives = len(set(recommended_at_k) & set(true_items))
    precision = true_positives / (len(recommended_at_k) if len(recommended_at_k) > 0 else 1)
    return precision

precisions = []
for i in range(len(predictions_df)):
    precisions.append(precision_at_k(predictions_df['recommended_items'][i], predictions_df['true_items'][i], k=5))

In [133]:
np.mean(precisions)

0.21351392509383427

In [134]:
## Recall
def recall_at_k(recommended_items, true_items, k):
    recommended_at_k = recommended_items if len(recommended_items) < k else recommended_items[:k]
    true_positives = len(set(recommended_at_k) & set(true_items))
    recall = true_positives / (len(true_items) if len(true_items) > 0 else 1)
    return recall

recalls = []
for i in range(len(predictions_df)):
    recalls.append(recall_at_k(predictions_df['recommended_items'][i], predictions_df['true_items'][i], k=5))

In [135]:
np.mean(recalls)

0.07003094334853205

## Tuning

In [153]:
def get_rules_df(baskets_tuples, support, confidence):
    itemsets, rules = apriori_efficient(baskets_tuples, min_support=support, min_confidence=confidence)

    len_transactions = len(baskets_tuples)
    itemsets_flattened = []
    itemsets_supports = []
    for i in itemsets.keys():
        itemsets_flattened.extend([item[0] for item in itemsets[i].items()])
        itemsets_supports.extend([item[1]/len_transactions for item in itemsets[i].items()])
    
    itemsets_dict = {
        'support': itemsets_supports,
        'itemsets': itemsets_flattened
    }
    itemsets_df = pd.DataFrame(itemsets_dict)

    rules = fp.association_rules(itemsets_df, metric="confidence", min_threshold=confidence)
    
    return rules

In [154]:
def get_predictions_from_rules(rules_df, baskets_test):
    predictions = {
    'user': [],
    'recommended_items': [],
    'true_items': []
    }

    for index, row in baskets_test.iterrows():
        user = row['user']
        train_items = row['games']
        test_items = row['test']
        recommended_items = recommender_association(rules, train_items, N=10)

        predictions['user'].append(user)
        predictions['recommended_items'].append(recommended_items)
        predictions['true_items'].append(test_items)
        
    predictions_df = pd.DataFrame(predictions)
    return predictions_df

In [155]:
def evaluate_model(predictions_df, k):
    precisions = []
    recalls = []
    for i in range(len(predictions_df)):
        precisions.append(precision_at_k(predictions_df['recommended_items'][i], predictions_df['true_items'][i], k=k))
        recalls.append(recall_at_k(predictions_df['recommended_items'][i], predictions_df['true_items'][i], k=k))

    return np.mean(precisions), np.mean(recalls)

In [ ]:
baskets_train_as_tuples = [tuple(row) for row in baskets_train['games']]
supports = [0.01, 0.005]
confidences = [0.6, 0.7, 0.8]
ks = [3, 5, 10]

evaluation_results = {
    'support': [],
    'confidence': [],
    'k': [],
    'precision': [],
    'recall': []
}

for support in supports:
    for confidence in confidences:
        rules_df = get_rules_df(baskets_train_as_tuples, support, confidence)
        pickle.dump(rules_df, open(f'data/freq_itemsets_rules_s{support}_c{confidence}.pkl', 'wb'))
        predictions_df = get_predictions_from_rules(rules_df, baskets_test)
        pickle.dump(predictions_df, open(f'data/freq_itemsets_pred_s{support}_c{confidence}.pkl', 'wb'))
        for k in ks:
            precision, recall = evaluate_model(predictions_df, k)
            print(f"Support: {support}, Confidence: {confidence}, K: {k} => Precision: {precision}, Recall: {recall}")
            evaluation_results['support'].append(support)
            evaluation_results['confidence'].append(confidence)
            evaluation_results['k'].append(k)
            evaluation_results['precision'].append(precision)
            evaluation_results['recall'].append(recall)
           
evaluation_results_df = pd.DataFrame(evaluation_results)
evaluation_results_df

Support: 0.01, Confidence: 0.6, K: 3 => Precision: 0.21192270635799632, Recall: 0.056718918961236806
Support: 0.01, Confidence: 0.6, K: 5 => Precision: 0.21179754540461207, Recall: 0.06964153339809682
Support: 0.01, Confidence: 0.6, K: 10 => Precision: 0.2115471913393251, Recall: 0.08013231130006615
Support: 0.01, Confidence: 0.7, K: 3 => Precision: 0.21192270635799632, Recall: 0.056718918961236806
Support: 0.01, Confidence: 0.7, K: 5 => Precision: 0.21179754540461207, Recall: 0.06964153339809682
Support: 0.01, Confidence: 0.7, K: 10 => Precision: 0.2115471913393251, Recall: 0.08013231130006615
Support: 0.01, Confidence: 0.8, K: 3 => Precision: 0.21192270635799632, Recall: 0.056718918961236806
Support: 0.01, Confidence: 0.8, K: 5 => Precision: 0.21179754540461207, Recall: 0.06964153339809682
Support: 0.01, Confidence: 0.8, K: 10 => Precision: 0.2115471913393251, Recall: 0.08013231130006615
Support: 0.005, Confidence: 0.6, K: 3 => Precision: 0.21192270635799632, Recall: 0.05671891896123